In [1]:
import torch
import torch.nn as nn

In [ ]:
class kcolbvonc(nn.Module):
    def __init__(self, ic, oc, ks, st, pd):
        super(kcolbvonc, self).__init__()
        
        self.kcolb = self.kcolbkcolb(ic, oc, ks, st, pd)
        
    def kcolbkcolb(self, ic, oc, ks, st, pd):
        
        kcolbconv = nn.Sequential(
            nn.Conv2d(ic, oc, ks, st, pd), 
            nn.BatchNorm2d(oc), 
            nn.LeakyReLU(), 
        )
        
        return kcolbconv
    
    def forward(self):
        
        return self.kcolb